Судебная практика https://sudact.ru/

В составе раздела – обзор решений судов по наиболее популярным правовым вопросам и ситуациям.

По административным делам - https://sudact.ru/practice/sudebnaya-praktika-po-administrativnym-delam/

По гражданским делам - https://sudact.ru/practice/sudebnaya-praktika-po-grazhdanskim-delam/

По уголовным делам - https://sudact.ru/practice/sudebnaya-praktika-po-ugolovnym-delam/

In [2]:
import numpy as np

In [3]:
sections = np.array([
    ('По административным делам', 'https://sudact.ru/practice/sudebnaya-praktika-po-administrativnym-delam/'),
    ('По гражданским делам', 'https://sudact.ru/practice/sudebnaya-praktika-po-grazhdanskim-delam/'),
    ('По уголовным делам', 'https://sudact.ru/practice/sudebnaya-praktika-po-ugolovnym-delam/')
])

In [4]:
sections[:, 1]

array(['https://sudact.ru/practice/sudebnaya-praktika-po-administrativnym-delam/',
       'https://sudact.ru/practice/sudebnaya-praktika-po-grazhdanskim-delam/',
       'https://sudact.ru/practice/sudebnaya-praktika-po-ugolovnym-delam/'],
      dtype='<U72')

1. Заходим на страницу раздела
2. Получаем список подразделов по применению конкретных норм
3. Получаем список решений 
4. Переходим на следующую страницу
5. Повторяем пункты 3, 4, пока есть страницы

Для конкретного решения у нас есть структурированная информация:
* название (содержит номер и дату)
* ссылка на страницу
* раздел
* подраздел
* суд
* html страницы

In [ ]:
#!pip install scrapy

In [ ]:
#!pip install pyopenssl==22.0.0

In [ ]:
#!pip install crochet

In [40]:
import sqlite3

In [147]:
con = sqlite3.connect("sudactru.db")

cur = con.cursor()

sql = '''
CREATE TABLE IF NOT EXISTS sections(
    id INTEGER PRIMARY KEY,
    name TEXT,
    href TEXT
)
'''

cur.executescript(sql)
con.commit()

In [148]:
#import pandas as pd

In [149]:
sql = '''
INSERT INTO sections(name, href) 
    VALUES (?, ?)
'''
cur.executemany(sql, sections)
con.commit()

In [150]:
res = cur.execute("SELECT * FROM sections;")
res.fetchall()

[(1,
  'По административным делам',
  'https://sudact.ru/practice/sudebnaya-praktika-po-administrativnym-delam/'),
 (2,
  'По гражданским делам',
  'https://sudact.ru/practice/sudebnaya-praktika-po-grazhdanskim-delam/'),
 (3,
  'По уголовным делам',
  'https://sudact.ru/practice/sudebnaya-praktika-po-ugolovnym-delam/')]

In [151]:
class SqliteSectionPipeline:

    def __init__(self):
        self.con = sqlite3.connect('sudactru.db')
        self.cur = self.con.cursor()
        self.cur.execute("""
        CREATE TABLE IF NOT EXISTS subsections(
            id INTEGER PRIMARY KEY,
            name TEXT,
            href TEXT UNIQUE,
            section_id INT,
            FOREIGN KEY (section_id) REFERENCES sections(id)
        )
        """)


    def process_item(self, item, spider):
        
        self.cur.execute("""
            INSERT INTO subsections (name, href, section_id) VALUES (?, ?, ?)
        """,
        (
            item['name'],
            item['href'],
            item['section_id']
        ))

        self.con.commit()

        return item


In [152]:
# scrape webpage
import scrapy
from scrapy.crawler import CrawlerRunner
# text cleaning
import re
# Reactor restart
from crochet import setup, wait_for
setup()

In [153]:
class SudactSectionSpider(scrapy.Spider):
    """
    Парсинг списка страниц на применения разных норм в рамках данного раздела.
    """
    def __init__(self):
        self.con = sqlite3.connect('sudactru.db')
        self.cur = self.con.cursor()
        
    name = "SudactSectionSpider"
    allowed_domains = ['sudact.ru']
    start_urls = list(sections[:, 1])
    
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.SqliteSectionPipeline': 1
        },
        "DOWNLOAD_DELAY": 0.25,
        "LOG_LEVEL": "WARNING",
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
    }

    def parse(self, response):
        current_url = response.request.url   
        h1 = response.xpath('//h1/text()').get()    
        print(h1)
        res = self.cur.execute("SELECT id FROM sections WHERE href = ?", (current_url,))
        section_id = res.fetchone()[0]
            
        for link in response.css('.wos'):
            yield {'name': link.css('::text').get(), 
                   'href': link.css('a').attrib["href"],
                   'section_id': section_id
                  }

@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(SudactSectionSpider)
    return d

In [154]:
run_spider()

Судебная практика по административным делам
Судебная практика по гражданским делам
Судебная практика по уголовным делам


In [234]:
res = cur.execute("SELECT href FROM subsections;")
list(map(lambda x: 'https://sudact.ru' + x[0], res.fetchall()))

['https://sudact.ru/practice/amnistiya/',
 'https://sudact.ru/practice/dokazatelstva/',
 'https://sudact.ru/practice/halatnost/',
 'https://sudact.ru/practice/inostrannye-grazhdane/',
 'https://sudact.ru/practice/kleveta/',
 'https://sudact.ru/practice/kommercheskij-podkup/',
 'https://sudact.ru/practice/kontrabanda/',
 'https://sudact.ru/practice/mery-presecheniya/',
 'https://sudact.ru/practice/narushenie-prav-invalidov/',
 'https://sudact.ru/practice/narushenie-pravil-dorozhnogo-dvizheniya/',
 'https://sudact.ru/practice/nezakonnoe-poluchenie-kredita/',
 'https://sudact.ru/practice/nezakonnoe-predprinimatelstvo/',
 'https://sudact.ru/practice/oskorblenie/',
 'https://sudact.ru/practice/osushestvlenie-predprinimatelskoj-deyatelnosti-bez/',
 'https://sudact.ru/practice/personalnye-dannye/',
 'https://sudact.ru/practice/po-delam-o-huliganstve/',
 'https://sudact.ru/practice/po-delam-ob-iznasilovanii/',
 'https://sudact.ru/practice/po-delam-ob-ubijstve/',
 'https://sudact.ru/practice/po

In [233]:
res = cur.execute("SELECT section_id, COUNT(*) FROM subsections GROUP BY section_id;")
res.fetchall()

[(1, 13), (2, 5), (3, 34)]

In [241]:
class SqliteSubSectionPipeline:
    
    def __init__(self):
        self.con = sqlite3.connect('sudactru.db')
        self.cur = self.con.cursor()
        self.cur.execute("""
        CREATE TABLE IF NOT EXISTS decisions(
            id INTEGER PRIMARY KEY,
            name TEXT,
            href TEXT UNIQUE,
            subsection_id INT,
            justice TEXT,
            short_descr TEXT,
            FOREIGN KEY (subsection_id) REFERENCES subsection(id)
        )
        """)
        # justice - название суда и типа дела, 
        # например "Шатурский городской суд (Московская область) - Гражданское"
        # short_descr - короткое "описание" решения, содержащие название нормы, 
        # в рамках которой рассматривалось данное дело


    def process_item(self, item, spider):
                
        self.cur.execute("""
            INSERT INTO decisions (
                name, 
                href, 
                subsection_id,
                justice,
                short_descr
            ) VALUES (?, ?, ?, ?, ?)
        """,
        (
            item['name'],
            item['href'],
            item['subsection_id'],
            item['justice'],
            item['short_descr']
        ))

        self.con.commit()
        
        return item

In [245]:
class SudactSubSectionSpider(scrapy.Spider):
    """
    Парсинг списка решений по данной правовой норме.
    """
    def __init__(self):
        self.con = sqlite3.connect('sudactru.db')
        self.cur = self.con.cursor()
        
    name = "SudactSubSectionSpider"
    allowed_domains = ['sudact.ru']
    
    res = cur.execute("SELECT href FROM subsections;")
    start_urls = list(map(lambda x: 'https://sudact.ru' + x[0], res.fetchall()))
    #start_urls = ['https://sudact.ru/practice/po-zashite-prav-potrebitelej/?page=49']
        
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.SqliteSubSectionPipeline': 1
        },
        "DOWNLOAD_DELAY": 0.25,
        "LOG_LEVEL": "WARNING",
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
    }

    def parse(self, response):
        current_url = response.request.url   
        h1 = response.xpath('//h1/text()').get()
        
        res = self.cur.execute("SELECT id FROM subsections WHERE href = ?", (
            current_url.replace('https://sudact.ru', '').split('?')[0],
        ))
        subsection_id = res.fetchone()[0]
           
        print(h1, current_url, subsection_id)
        
        for li in response.css('.results li'):
            yield {'name': li.css('a').css('::text').get(), 
                   'href': li.css('a').attrib["href"],
                   'subsection_id': subsection_id,
                   'justice': li.css('.b-justice::text').get(),
                   'short_descr': ''.join(li.css('::text').getall()[3:])
                  }
        
        next_page = response.css('.page-next a')
        if len(next_page) > 0:
            yield scrapy.Request(
                response.urljoin(current_url.split('?')[0] + next_page.attrib["href"]),
                callback=self.parse
            )

@wait_for(None)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(SudactSubSectionSpider)
    return d

In [246]:
run_spider()

Амнистия - судебная практика https://sudact.ru/practice/amnistiya/ 32
Доказательства - судебная практика https://sudact.ru/practice/dokazatelstva/ 51
Халатность - судебная практика https://sudact.ru/practice/halatnost/ 35
Иностранные граждане - судебная практика https://sudact.ru/practice/inostrannye-grazhdane/ 8
Клевета - судебная практика https://sudact.ru/practice/kleveta/ 19
Коммерческий подкуп - судебная практика https://sudact.ru/practice/kommercheskij-podkup/ 29
Контрабанда - судебная практика https://sudact.ru/practice/kontrabanda/ 40
Меры пресечения - судебная практика https://sudact.ru/practice/mery-presecheniya/ 52
Нарушение прав инвалидов - судебная практика https://sudact.ru/practice/narushenie-prav-invalidov/ 3
Нарушение правил дорожного движения - судебная практика https://sudact.ru/practice/narushenie-pravil-dorozhnogo-dvizheniya/ 49
Незаконное получение кредита - судебная практика https://sudact.ru/practice/nezakonnoe-poluchenie-kredita/ 4
Незаконное предпринимательств

Персональные данные - судебная практика https://sudact.ru/practice/personalnye-dannye/?page=5 7
Судебная практика по делам о хулиганстве https://sudact.ru/practice/po-delam-o-huliganstve/?page=5 31
Судебная практика по делам об изнасиловании https://sudact.ru/practice/po-delam-ob-iznasilovanii/?page=5 26
Амнистия - судебная практика https://sudact.ru/practice/amnistiya/?page=6 32
Доказательства - судебная практика https://sudact.ru/practice/dokazatelstva/?page=6 51
Халатность - судебная практика https://sudact.ru/practice/halatnost/?page=6 35
Иностранные граждане - судебная практика https://sudact.ru/practice/inostrannye-grazhdane/?page=6 8
Клевета - судебная практика https://sudact.ru/practice/kleveta/?page=6 19
Коммерческий подкуп - судебная практика https://sudact.ru/practice/kommercheskij-podkup/?page=6 29
Судебная практика по делам об убийстве https://sudact.ru/practice/po-delam-ob-ubijstve/?page=2 21
Меры пресечения - судебная практика https://sudact.ru/practice/mery-presecheniya

Контрабанда - судебная практика https://sudact.ru/practice/kontrabanda/?page=9 40
Незаконное предпринимательство - судебная практика https://sudact.ru/practice/nezakonnoe-predprinimatelstvo/?page=10 41
Оскорбление - судебная практика https://sudact.ru/practice/oskorblenie/?page=10 2
Осуществление предпринимательской деятельности без регистрации или без разрешения - судебная практика https://sudact.ru/practice/osushestvlenie-predprinimatelskoj-deyatelnosti-bez/?page=10 5
Доказательства - судебная практика https://sudact.ru/practice/dokazatelstva/?page=10 51
Судебная практика по делам о хулиганстве https://sudact.ru/practice/po-delam-o-huliganstve/?page=10 31
Судебная практика по делам об изнасиловании https://sudact.ru/practice/po-delam-ob-iznasilovanii/?page=10 26
Амнистия - судебная практика https://sudact.ru/practice/amnistiya/?page=11 32
Персональные данные - судебная практика https://sudact.ru/practice/personalnye-dannye/?page=10 7
Халатность - судебная практика https://sudact.ru/p

Клевета - судебная практика https://sudact.ru/practice/kleveta/?page=15 19
Персональные данные - судебная практика https://sudact.ru/practice/personalnye-dannye/?page=14 7
Судебная практика по делам об убийстве https://sudact.ru/practice/po-delam-ob-ubijstve/?page=11 21
Меры пресечения - судебная практика https://sudact.ru/practice/mery-presecheniya/?page=15 52
Коммерческий подкуп - судебная практика https://sudact.ru/practice/kommercheskij-podkup/?page=15 29
Нарушение прав инвалидов - судебная практика https://sudact.ru/practice/narushenie-prav-invalidov/?page=13 3
Контрабанда - судебная практика https://sudact.ru/practice/kontrabanda/?page=14 40
Незаконное предпринимательство - судебная практика https://sudact.ru/practice/nezakonnoe-predprinimatelstvo/?page=15 41
Оскорбление - судебная практика https://sudact.ru/practice/oskorblenie/?page=15 2
Осуществление предпринимательской деятельности без регистрации или без разрешения - судебная практика https://sudact.ru/practice/osushestvleni

Судебная практика по делам об изнасиловании https://sudact.ru/practice/po-delam-ob-iznasilovanii/?page=19 26
Контрабанда - судебная практика https://sudact.ru/practice/kontrabanda/?page=18 40
Нарушение правил дорожного движения - судебная практика https://sudact.ru/practice/narushenie-pravil-dorozhnogo-dvizheniya/?page=19 49
Халатность - судебная практика https://sudact.ru/practice/halatnost/?page=20 35
Иностранные граждане - судебная практика https://sudact.ru/practice/inostrannye-grazhdane/?page=20 8
Клевета - судебная практика https://sudact.ru/practice/kleveta/?page=20 19
Доказательства - судебная практика https://sudact.ru/practice/dokazatelstva/?page=19 51
Судебная практика по делам об убийстве https://sudact.ru/practice/po-delam-ob-ubijstve/?page=16 21
Меры пресечения - судебная практика https://sudact.ru/practice/mery-presecheniya/?page=20 52
Коммерческий подкуп - судебная практика https://sudact.ru/practice/kommercheskij-podkup/?page=20 29
Нарушение прав инвалидов - судебная п

...

Самоуправство - судебная практика https://sudact.ru/practice/samoupravstvo/?page=18 20
Приговор, неисполнение приговора - судебная практика https://sudact.ru/practice/prigovor-neispolnenie-prigovora/?page=20 50
Соучастие, предварительный сговор - судебная практика https://sudact.ru/practice/souchastie-predvaritelnyj-sgovor/?page=18 45
Умышленное причинение тяжкого вреда здоровью - судебная практика https://sudact.ru/practice/umyshlennoe-prichinenie-tyazhkogo-vreda-zdorovyu/?page=16 47
Уголовная ответственность несовершеннолетних - судебная практика https://sudact.ru/practice/ugolovnaya-otvetstvennost-nesovershennoletnih/?page=16 30
Разбой - судебная практика https://sudact.ru/practice/razboj/?page=19 28
Злоупотребление должностными полномочиями – судебная практика https://sudact.ru/practice/zloupotreblenie-dolzhnostnymi-polnomochiyami/?page=16 22
Присвоение и растрата - судебная практика https://sudact.ru/practice/prisvoenie-i-rastrata/?page=20 25
Преступное сообщество - судебная практ

Злоупотребление должностными полномочиями – судебная практика https://sudact.ru/practice/zloupotreblenie-dolzhnostnymi-polnomochiyami/?page=22 22
Превышение должностных полномочий - судебная практика https://sudact.ru/practice/prevyshenie-dolzhnostnyh-polnomochij/?page=27 37
Присвоение и растрата - судебная практика https://sudact.ru/practice/prisvoenie-i-rastrata/?page=26 25
Уголовная ответственность несовершеннолетних - судебная практика https://sudact.ru/practice/ugolovnaya-otvetstvennost-nesovershennoletnih/?page=22 30
Преступное сообщество - судебная практика https://sudact.ru/practice/prestupnoe-soobshestvo/?page=43 46
Самоуправство - судебная практика https://sudact.ru/practice/samoupravstvo/?page=25 20
Приговор, неисполнение приговора - судебная практика https://sudact.ru/practice/prigovor-neispolnenie-prigovora/?page=27 50
Соучастие, предварительный сговор - судебная практика https://sudact.ru/practice/souchastie-predvaritelnyj-sgovor/?page=25 45
Умышленное причинение тяжкого 

Разбой - судебная практика https://sudact.ru/practice/razboj/?page=32 28
Злоупотребление должностными полномочиями – судебная практика https://sudact.ru/practice/zloupotreblenie-dolzhnostnymi-polnomochiyami/?page=29 22
Превышение должностных полномочий - судебная практика https://sudact.ru/practice/prevyshenie-dolzhnostnyh-polnomochij/?page=34 37
Присвоение и растрата - судебная практика https://sudact.ru/practice/prisvoenie-i-rastrata/?page=33 25
Самоуправство - судебная практика https://sudact.ru/practice/samoupravstvo/?page=32 20
Уголовная ответственность несовершеннолетних - судебная практика https://sudact.ru/practice/ugolovnaya-otvetstvennost-nesovershennoletnih/?page=29 30
Преступное сообщество - судебная практика https://sudact.ru/practice/prestupnoe-soobshestvo/?page=50 46
Приговор, неисполнение приговора - судебная практика https://sudact.ru/practice/prigovor-neispolnenie-prigovora/?page=34 50
Соучастие, предварительный сговор - судебная практика https://sudact.ru/practice/so

Самоуправство - судебная практика https://sudact.ru/practice/samoupravstvo/?page=39 20
Приговор, неисполнение приговора - судебная практика https://sudact.ru/practice/prigovor-neispolnenie-prigovora/?page=41 50
Соучастие, предварительный сговор - судебная практика https://sudact.ru/practice/souchastie-predvaritelnyj-sgovor/?page=39 45
Уголовная ответственность несовершеннолетних - судебная практика https://sudact.ru/practice/ugolovnaya-otvetstvennost-nesovershennoletnih/?page=36 30
Умышленное причинение тяжкого вреда здоровью - судебная практика https://sudact.ru/practice/umyshlennoe-prichinenie-tyazhkogo-vreda-zdorovyu/?page=37 47
Разбой - судебная практика https://sudact.ru/practice/razboj/?page=40 28
Злоупотребление должностными полномочиями – судебная практика https://sudact.ru/practice/zloupotreblenie-dolzhnostnymi-polnomochiyami/?page=37 22
Превышение должностных полномочий - судебная практика https://sudact.ru/practice/prevyshenie-dolzhnostnyh-polnomochij/?page=42 37
Самоуправст

Разбой - судебная практика https://sudact.ru/practice/razboj/?page=47 28
Злоупотребление должностными полномочиями – судебная практика https://sudact.ru/practice/zloupotreblenie-dolzhnostnymi-polnomochiyami/?page=44 22
Превышение должностных полномочий - судебная практика https://sudact.ru/practice/prevyshenie-dolzhnostnyh-polnomochij/?page=49 37
Самоуправство - судебная практика https://sudact.ru/practice/samoupravstvo/?page=47 20
Присвоение и растрата - судебная практика https://sudact.ru/practice/prisvoenie-i-rastrata/?page=48 25
Приговор, неисполнение приговора - судебная практика https://sudact.ru/practice/prigovor-neispolnenie-prigovora/?page=49 50
Соучастие, предварительный сговор - судебная практика https://sudact.ru/practice/souchastie-predvaritelnyj-sgovor/?page=47 45
Умышленное причинение тяжкого вреда здоровью - судебная практика https://sudact.ru/practice/umyshlennoe-prichinenie-tyazhkogo-vreda-zdorovyu/?page=45 47
Уголовная ответственность несовершеннолетних - судебная пр

In [269]:
res = cur.execute("SELECT COUNT(*) FROM decisions;")
res.fetchall()

[(51304,)]

In [270]:
res = cur.execute("SELECT subsection_id, COUNT(*) FROM decisions GROUP BY subsection_id;")
res.fetchall()

[(1, 1000),
 (2, 1270),
 (3, 744),
 (4, 734),
 (5, 1290),
 (6, 1000),
 (7, 1280),
 (8, 1310),
 (9, 1000),
 (10, 1000),
 (11, 1000),
 (12, 1000),
 (13, 1040),
 (14, 22),
 (15, 20),
 (16, 90),
 (17, 22),
 (18, 12),
 (19, 1310),
 (20, 1000),
 (21, 1270),
 (22, 1000),
 (23, 1000),
 (24, 1000),
 (25, 1000),
 (26, 1300),
 (27, 1000),
 (28, 1000),
 (29, 1300),
 (30, 1000),
 (31, 1290),
 (32, 1300),
 (33, 1000),
 (34, 1000),
 (35, 1290),
 (36, 1000),
 (37, 1000),
 (38, 1000),
 (39, 1000),
 (40, 1270),
 (41, 1300),
 (42, 1000),
 (43, 1000),
 (44, 1000),
 (45, 1000),
 (46, 1000),
 (47, 1000),
 (48, 1000),
 (49, 1290),
 (50, 1000),
 (51, 1260),
 (52, 1290)]

In [271]:
res = cur.execute("SELECT * FROM decisions LIMIT 1;")
res.fetchall()

[(1,
  'Приговор № 1-13/2022 от 17 февраля 2022 г. по делу № 1-13/2022',
  '/regular/doc/chaeDyxsKiYN/',
  32,
  'Шумихинский районный суд (Курганская область) - Уголовное\n\n\n',
  '\n\n...Федерации" решение суда об изменении категории преступления с тяжкого на преступление средней тяжести позволяет суду при наличии оснований, предусмотренных статьями 75, 76, 78, 80.1,   84 ,  92, 94 УК РФ, освободить осужденного от отбывания назначенного наказания. Лицо, освобожденное от отбывания наказания по указанным основаниям, считается несудимым (часть 2 статьи 86 УК РФ).\nВ ...\n\n\n')]

In [ ]:
class SqliteDecisionPipeline:
    
    def __init__(self):
        self.con = sqlite3.connect('sudactru.db')
        self.cur = self.con.cursor()
        self.cur.execute("""
        CREATE TABLE IF NOT EXISTS decisions__data(
            id INTEGER PRIMARY KEY,
            name TEXT UNIQUE,
            value TEXT,
            decision_id INT
            FOREIGN KEY (decision_id) REFERENCES decisions(id)
        )
        """)


    def process_item(self, item, spider):
                
        self.cur.execute("""
            INSERT INTO decisions__data (
                name, 
                value
            ) VALUES (?, ?)
        """,
        (
            item['name'],
            item['value'],
            item['decision_id']
        ))

        #self.con.commit()
        
        return item

In [254]:
class DecisionClearing(object):
    def process_item(self, item, spider):
        # удаляем рекламу
        # удаляем скрипты
        # html переносы заменяем на обычные (текстовые)
        # удаляем остальные html теги        
        
        print(item)
        
        item['value'] = self.__remove_html_tags__(item['value'])
        return item
        # .css('::text').getall()
#         lines = dict(item)["quote"].splitlines()
#         first_line = self.__remove_html_tags__(lines[0])
#         return {'quote': first_line}

    def __remove_html_tags__(self, text):
        """remove html tags from string"""
        html_tags = re.compile('<.*?>')
        return re.sub(html_tags, '', text)

In [264]:
class SudactDecisionSpider(scrapy.Spider):
    """
    Парсинг содержимого решений в формате html 
    """
    def __init__(self):
        self.con = sqlite3.connect('sudactru.db')
        self.cur = self.con.cursor()
        
    name = "SudactDecisionSpider"
    allowed_domains = ['sudact.ru']
    
    res = cur.execute("SELECT href FROM decisions;")
    start_urls = list(map(lambda x: 'https://sudact.ru' + x[0], res.fetchall()))[:1]
        
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.DecisionClearing': 1,
            '__main__.SqliteSubSectionPipeline': 2
        },
        "DOWNLOAD_DELAY": 0.25,
        "LOG_LEVEL": "WARNING",
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
    }

    def parse(self, response):
        
        current_url = response.request.url   
        h1 = response.xpath('//h1/text()').get()
        
        res = self.cur.execute("SELECT id FROM decisions WHERE href = ?", (
            current_url.replace('https://sudact.ru', ''),
        ))
        decision_id = res.fetchone()[0]
           
        print(h1, current_url, subsection_id)
        
        html = response.css('body > div.h-wrap > div.h-wrap-fixed > div.h-content-wrap2 > table > tbody > tr > td.h-col1.h-col1-inner3')
        print(html)
        
        yield {'name': 'content', 
               'value': html,
               'decision_id': decision_id
              }

@wait_for(None)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(SudactDecisionSpider)
    return d

Судебная практика по лишению прав за обгон, выезд на встречную полосу https://sudact.ru/practice/po-lisheniyu-prav-za-obgon-vstrechku/ 11
Контрабанда - судебная практика https://sudact.ru/practice/kontrabanda/?page=45 40
Судебная практика по лишению прав за «пьянку» (управление ТС в состоянии опьянения, отказ от медицинского освидетельствования) https://sudact.ru/practice/po-lisheniyu-prav-za-pyanku-upravlenie-ts-v-sostoy/ 10
Судебная практика по грабежам https://sudact.ru/practice/po-grabezham/?page=2 33
Судебная практика по мошенничеству https://sudact.ru/practice/po-moshennichestvu/ 24
Судебная практика по ДТП (причинение легкого или средней тяжести вреда здоровью потерпевшего) https://sudact.ru/practice/po-dtp-prichinenie-legkogo-ili-srednej-tyazhesti-v/?page=26 12
Амнистия - судебная практика https://sudact.ru/practice/amnistiya/?page=49 32
Судебная практика по коррупционным преступлениям, по взяточничеству https://sudact.ru/practice/po-korrupcionnym-prestupleniyam-po-vzyatochnich

Коммерческий подкуп - судебная практика https://sudact.ru/practice/kommercheskij-podkup/?page=47 29
Судебная практика по лишению прав за «пьянку» (управление ТС в состоянии опьянения, отказ от медицинского освидетельствования) https://sudact.ru/practice/po-lisheniyu-prav-za-pyanku-upravlenie-ts-v-sostoy/?page=5 10
Судебная практика по грабежам https://sudact.ru/practice/po-grabezham/?page=6 33
Судебная практика по мошенничеству https://sudact.ru/practice/po-moshennichestvu/?page=5 24
Судебная практика по ДТП (причинение легкого или средней тяжести вреда здоровью потерпевшего) https://sudact.ru/practice/po-dtp-prichinenie-legkogo-ili-srednej-tyazhesti-v/?page=30 12
Персональные данные - судебная практика https://sudact.ru/practice/personalnye-dannye/?page=50 7
Судебная практика по коррупционным преступлениям, по взяточничеству https://sudact.ru/practice/po-korrupcionnym-prestupleniyam-po-vzyatochnichest/?page=6 34
Судебная практика по наркотикам https://sudact.ru/practice/po-narkotikam/

Судебная практика по грабежам https://sudact.ru/practice/po-grabezham/?page=10 33
Судебная практика по мошенничеству https://sudact.ru/practice/po-moshennichestvu/?page=9 24
Судебная практика по ДТП (причинение легкого или средней тяжести вреда здоровью потерпевшего) https://sudact.ru/practice/po-dtp-prichinenie-legkogo-ili-srednej-tyazhesti-v/?page=34 12
Судебная практика по охране труда https://sudact.ru/practice/po-ohrane-truda/?page=7 36
Судебная практика по коррупционным преступлениям, по взяточничеству https://sudact.ru/practice/po-korrupcionnym-prestupleniyam-po-vzyatochnichest/?page=10 34
Судебная практика по наркотикам https://sudact.ru/practice/po-narkotikam/?page=9 38
Судебная практика по нарушениям правил дорожного движения https://sudact.ru/practice/po-narusheniyam-pdd/?page=8 9
Судебная практика по поджогам https://sudact.ru/practice/po-podzhogam/?page=7 42
Судебная практика по ДТП (невыполнение требований при дорожно-транспортном происшествии) https://sudact.ru/practice/

Судебная практика по мошенничеству https://sudact.ru/practice/po-moshennichestvu/?page=13 24
Судебная практика по ДТП (причинение легкого или средней тяжести вреда здоровью потерпевшего) https://sudact.ru/practice/po-dtp-prichinenie-legkogo-ili-srednej-tyazhesti-v/?page=38 12
Судебная практика по охране труда https://sudact.ru/practice/po-ohrane-truda/?page=11 36
Судебная практика по коррупционным преступлениям, по взяточничеству https://sudact.ru/practice/po-korrupcionnym-prestupleniyam-po-vzyatochnichest/?page=14 34
Судебная практика по наркотикам https://sudact.ru/practice/po-narkotikam/?page=13 38
Судебная практика по нарушениям правил дорожного движения https://sudact.ru/practice/po-narusheniyam-pdd/?page=12 9
Судебная практика по поджогам https://sudact.ru/practice/po-podzhogam/?page=11 42
Судебная практика по ДТП (невыполнение требований при дорожно-транспортном происшествии) https://sudact.ru/practice/po-dtp-nevypolnenie-trebovanij-pri-dtp/?page=41 13
Похищение - судебная практ

In [258]:
run_spider()

Амнистия - судебная практика https://sudact.ru/practice/amnistiya/ 32
Доказательства - судебная практика https://sudact.ru/practice/dokazatelstva/ 51
Халатность - судебная практика https://sudact.ru/practice/halatnost/ 35
Иностранные граждане - судебная практика https://sudact.ru/practice/inostrannye-grazhdane/ 8
Клевета - судебная практика https://sudact.ru/practice/kleveta/ 19
Коммерческий подкуп - судебная практика https://sudact.ru/practice/kommercheskij-podkup/ 29
Контрабанда - судебная практика https://sudact.ru/practice/kontrabanda/ 40
Меры пресечения - судебная практика https://sudact.ru/practice/mery-presecheniya/ 52
Нарушение прав инвалидов - судебная практика https://sudact.ru/practice/narushenie-prav-invalidov/ 3
Нарушение правил дорожного движения - судебная практика https://sudact.ru/practice/narushenie-pravil-dorozhnogo-dvizheniya/ 49
Незаконное получение кредита - судебная практика https://sudact.ru/practice/nezakonnoe-poluchenie-kredita/ 4


KeyboardInterrupt: 

In [184]:
# res = cur.execute("SELECT href FROM subsections;")
# res.fetchall()

In [185]:
# current_url = 'https://sudact.ru/practice/amnistiya/'

In [186]:
# current_url.split('?')[0]

In [187]:
# res = cur.execute("SELECT id FROM subsections WHERE href = ?", (current_url.replace('https://sudact.ru', '').split('?')[0],))
# subsection_id = res.fetchone()
# print(subsection_id)

In [188]:
# 'https://sudact.ru/practice/po-zashite-prav-potrebitelej/?page=5'.split('?')[0]

In [114]:
#cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
#print(cur.fetchall())

In [82]:
# res = cur.execute("SELECT id FROM sections WHERE href = ?", ('https://sudact.ru/practice/sudebnaya-praktika-po-ugolovnym-delam/',))
# section_id = res.fetchone()
# section_id

In [ ]:
            
# class ExtractFirstLine(object):
#     def process_item(self, item, spider):
#         """text processing"""
#         lines = dict(item)["quote"].splitlines()
#         #print(lines)
#         first_line = self.__remove_html_tags__(lines[0])

#         return {'quote': first_line}

#     def __remove_html_tags__(self, text):
#         """remove html tags from string"""
#         html_tags = re.compile('<.*?>')
#         return re.sub(html_tags, '', text)